# Import libraries

In [66]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv

# Get Cache

# Concurrency

# Get Sessions

In [67]:
session = requests.Session()

# Get Urls

In [68]:
def get_page_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@id="uc-btn-accept-banner"]').click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    s_links = soup.select('li.search-results__item a')

    url_list = []
    for element in s_links:
        link = element["href"]
        if '/agency' in link or '/en/classified/' not in link: 
            pass

        elif 'https://www.immoweb.be' not in link:
            link = 'https://www.immoweb.be' + link
            url_list.append(link)
        else:
            url_list.append(link)
    return url_list


def get_element_url_list():
    all_urls = []
    for i,x in enumerate(range(1,15)):
        urls = get_page_links(f'https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={x}&orderBy=relevance')
        for url in urls:
            start = time.time()
            if url not in all_urls:
                all_urls.append(url)
            end = time.time()
        print('got Url',i,end-start)
    print(len(all_urls))
    return all_urls

#get houses
#get appartsments

In [69]:
url_list = get_element_url_list()

got Url 0 1.430511474609375e-06
got Url 1 1.6689300537109375e-06
got Url 2 2.384185791015625e-06
got Url 3 3.337860107421875e-06
got Url 4 3.5762786865234375e-06
got Url 5 2.384185791015625e-06
got Url 6 5.245208740234375e-06
got Url 7 5.4836273193359375e-06
got Url 8 6.4373016357421875e-06
got Url 9 6.4373016357421875e-06
got Url 10 6.67572021484375e-06
got Url 11 7.62939453125e-06
got Url 12 9.298324584960938e-06
got Url 13 1.0251998901367188e-05
423


# Get Elements_properties_raw

In [70]:
def get_property(url):
    req_element = session.get(url)
    soup = BeautifulSoup(req_element.text, 'html.parser')
    element_info = soup.select('div.classified script')
    text = element_info[0].text

    return json.loads("".join(("".join(text.split('=')[1:]).split(";")[:-1])))

In [71]:
main_dictionary = dict()
start = time.time()
for index,url in enumerate(url_list):
    start = time.time()
    main_dictionary[url] = get_property(url)
    end = time.time()
    print('scraped',index,' time ',end-start,url)
end = time.time()
print(end-start)

scraped 0  time  0.5039529800415039 https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=63393bc012991
scraped 1  time  0.7473785877227783 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63393bc012991
scraped 2  time  0.30532360076904297 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63393bc012991
scraped 3  time  0.33572936058044434 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63393bc012991
scraped 4  time  0.6571512222290039 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63393bc012991
scraped 5  time  0.3137226104736328 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63393bc012991
scraped 6  time  0.31356000900268555 https://www.immoweb.be/en/classified/apartment/for-sa

In [72]:
for index,url in enumerate(url_list):
    print(index,url)

0 https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=63393bc012991
1 https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63393bc012991
2 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63393bc012991
3 https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63393bc012991
4 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63393bc012991
5 https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63393bc012991
6 https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=63393bc012991
7 https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=63393bc012991
8 https://www.immoweb.be/en/classified/new-real-estate-project-a

# Save into json

In [73]:
with open("Main_datas.json","w") as main_data_file:
    main_data_file.write(json.dumps(main_dictionary))

# Get Elements_properties_list

In [74]:
data_main_list = list()
start = time.time()
for index,element in enumerate(main_dictionary.values()):
    element_data = {
    'id': element['id'],
    'url': list(main_dictionary.keys())[index],
    'Locality': element['property']['location']['postalCode'] if element['property']['location'] is not None else None,
    'Type of property': element['property']['type'] if element['property']['type'] is not None else None,
    'Subtype of property': element['property']['subtype'] if element['property']['subtype'] is not None else None,
    'Price': element['price']['mainValue'] if element['price'] is not None else None,
    'Type of sale': element['price']['type'] if element['price'] is not None else None,
    'Number of rooms': element['property']['roomCount'] if element['property']  is not None else None,
    'Area': element['property']['netHabitableSurface'] if element['property'] is not None else None,
    'Fully equipped kitchen': element['property']['kitchen']['type'] if element['property']['kitchen'] is not None else None,
    'Furnished': element['transaction']['sale']['isFurnished'] if element['transaction']['sale'] is not None else None,
    'Open fire': element['property']['fireplaceExists'] if element['property'] is not None else None,
    'Terrace': element['property']['hasTerrace'] if element['property'] is not None else None,
    'Terrace area': element['property']['terraceSurface'] if element['property'] is not None else None,
    'Garden': element['property']['hasGarden'] if element['property'] is not None else None,
    'Garden area': element['property']['gardenSurface'] if element['property'] is not None else None,
    'Surface of the land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Surface area of the plot of land': element['property']['land']['surface'] if element['property']['land'] is not None else None,
    'Number of facades ': element['property']['building']['facadeCount'] if element['property']['building'] is not None else 1,
    'Swimming pool': element['property']['hasSwimmingPool'] if element['property'] is not None else None,
    'State of the building': element['property']['building']['condition'] if element['property']['building'] is not None else None
    }
    data_main_list.append(element_data)
    print('Done',list(main_dictionary.keys())[index])
    
end = time.time()
print(end-start)

Done https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/gent-sint-denijs-westrem/9051/10145286?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/10141214?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/brussels-city/1000/9881253?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/apartment/for-sale/laeken/1020/10143541?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/apartment/for-sale/hoboken/2660/10143954?searchId=63393bc012991
Done https://www.immoweb.be/en/classified/new-real-estate-project-apartments/for-sale/zedelgem/8210/10074861?searchId=63393bc012991
Done https://www.immoweb.be/en/classifie

In [75]:
print (data_main_list)

[{'id': 10146186, 'url': 'https://www.immoweb.be/en/classified/apartment/for-sale/etterbeek/1040/10146186?searchId=63393bc012991', 'Locality': '1040', 'Type of property': 'APARTMENT', 'Subtype of property': 'APARTMENT', 'Price': 275000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 75, 'Fully equipped kitchen': 'INSTALLED', 'Furnished': False, 'Open fire': True, 'Terrace': True, 'Terrace area': 9, 'Garden': None, 'Garden area': None, 'Surface of the land': None, 'Surface area of the plot of land': None, 'Number of facades ': 2, 'Swimming pool': None, 'State of the building': 'TO_BE_DONE_UP'}, {'id': 10145269, 'url': 'https://www.immoweb.be/en/classified/apartment/for-sale/woluwe-saint-lambert/1200/10145269?searchId=63393bc012991', 'Locality': '1200', 'Type of property': 'APARTMENT', 'Subtype of property': 'APARTMENT', 'Price': 275000, 'Type of sale': 'residential_sale', 'Number of rooms': None, 'Area': 83, 'Fully equipped kitchen': 'HYPER_EQUIPPED', 'Furnished': 

# Save information into CSV

In [113]:
import os
def save_to_csv(list_dict):
    """a function that prints a csv starting from a dictionarires list 
        main(list_dict) """
    # if the csv file doesn't exist, we create one
    dir_list = os.listdir()
    if "df_immo.csv" not in dir_list:
        # we need at least 2 dict to create a csv file, so I made a df from 2 dict (the first 2 dict that I have)
        df = pd.DataFrame([list_dict[0],list_dict[1]])
        df.to_csv("df_immo.csv", index=False)
    else:
        pass
    # to add every new dict to the csv
    for i in  range(2, len(data_main_list)): #len(data_main_list)
        df = pd.read_csv("df_immo.csv")
        #I don't know if we are already checking for duplicates, but this will do that:
        if data_main_list[i]["id"] not in df["id"].array:
            #open csv in append mode ("a")
            file = open('df_immo.csv', "a")
            #use csv library to add a row to the csv file. The row has to be in a list format, so I've used the .values() method of the ditionary class
            csv.writer(file).writerow(data_main_list[i].values())
            file.close  
        else:
            pass        
    #at the end close the csv file
    

It took 2,68 seconds for 400 houses, should take around 67 seconds for 10,000

In [118]:
start = time.time()
for i in range(25):    
    save_to_csv(data_main_list)
end = time.time()
print(end-start)

56.752532720565796


Checking if the file exists 4:36

not Checking if the file exists 0:53

Checking from a premade list 4:14

Checking from a premade list 0:59